In [1]:
!pip install deep-sort-realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 64.9 MB/s eta 0:00:00:00:010:01


In [2]:
!pip install torchsummary

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch

from torch.utils.data import DataLoader
import torch.nn as nn
from torch import flatten

from sklearn.metrics import accuracy_score, f1_score
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from datasets import load_dataset
from sklearn.model_selection import train_test_split

import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchsummary import summary
import cv2
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

cuda


# Faster RCNN

In [4]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2
from PIL import Image

fasterrcnn=fasterrcnn_resnet50_fpn_v2(pretrained=True).to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_V2_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth
100%|██████████| 167M/167M [00:01<00:00, 156MB/s]  


# Inference On Image

In [ ]:
fasterrcnn.eval()
car_index = 3 
filename = '/kaggle/input/car-dataset/img.jpg'
img = Image.open(filename) 
transform = transforms.ToTensor()
img_tensor = transform(img).to(device)
with torch.no_grad():
  pred = fasterrcnn([img_tensor])
pred[0].keys()
bounding_boxes,labels,confidences = pred[0]["boxes"],pred[0]["labels"],pred[0]["scores"]
labels,confidences

In [ ]:
indices = torch.argwhere(confidences>0.9)
indices=indices.cpu().numpy().flatten()
indices

In [ ]:
import matplotlib.patches as patches

bounding_boxes1,labels1,confidences1 = bounding_boxes.cpu().numpy(), labels.cpu().numpy(), confidences.cpu().numpy()

fig, ax = plt.subplots(1)

# Plot the image
ax.imshow(img)

# Iterate over bounding boxes and plot them with labels
for i in indices:
    # Get coordinates
    if(labels1[i].item()==car_index):
        x, y, w, h = bounding_boxes1[i]

        # Create a Rectangle patch
        rect = patches.Rectangle((x, y), w - x, h - y, linewidth=1, edgecolor='r', facecolor='none')

        # Add the patch to the Axes
        ax.add_patch(rect)

        # Add label and confidence score
        ax.text(x, y, f'{labels1[i].item()} - {confidences1[i].item():.2f}', fontsize=8, color='r')

# Show the plot
plt.show()

# Bounding Box Prediction On Video

In [ ]:
video = cv2.VideoCapture('/kaggle/input/traffic-junction/Images/Brigade Road/video_20240211_133745.mp4')

In [ ]:
def inferVideo(model,video, car_index):
    model.eval()
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter('/kaggle/working/brigadeRoad.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 15, (width, height))

    
    while video.isOpened():
        # Read the next frame
        ret, frame = video.read()
        # Check EOF
        if not ret:
            break        
        transform = transforms.ToTensor()
        img = transform(frame).to(device)
        with torch.no_grad():
          pred = model([img])

        bounding_box,labels,confidence = pred[0]["boxes"],pred[0]["labels"],pred[0]["scores"]
        indices = torch.argwhere(confidence>0.9)
        indices=indices.flatten()
        font = cv2.FONT_HERSHEY_COMPLEX
        bounding_box1=bounding_box.cpu().numpy()
        labels1=labels.cpu().numpy()

        if(len(indices)!=0):
            for i in indices:
                if(labels1[i] == car_index):
                    x1,y1,x2,y2 = bounding_box1[i].astype('int')
                    cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
                    cv2.putText(frame,"car",(x1,y1),font,0.5,(255,0,0),cv2.LINE_AA)

        out.write(frame)

    # Release the video capture object and close the window
    out.release()


In [ ]:
inferVideo(fasterrcnn,video,3)

# Faster RCNN + DEEPSORT

In [34]:
from deep_sort_realtime.deepsort_tracker import DeepSort 

video = cv2.VideoCapture('/kaggle/input/traffic-junction/Images/Images/Vidhana Soudha/video_20240211_142947.mp4')

In [35]:
object_tracker = DeepSort(max_iou_distance=0.2,max_age=20,nms_max_overlap = 0.05,gating_only_position=True,n_init=2,max_cosine_distance=0.9)

In [36]:
def TrackCar(model,video, car_index,object_tracker,out_path):
    model.eval()
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), 15, (width, height))
    setIDs=set()
    
    while video.isOpened():
        # Read the next frame
        ret, frame = video.read()
        if not ret:
            break
            
        transform = transforms.ToTensor()
        img = transform(frame).to(device)
        with torch.no_grad():
          pred = model([img])
        

        #Creating Detection data for cars to pass into SORT tracks directly 
        
        bounding_box,labels,confidence = pred[0]["boxes"],pred[0]["labels"],pred[0]["scores"]
        indices = torch.argwhere(confidence>0.9)
        indices=indices.flatten()
        font = cv2.FONT_HERSHEY_COMPLEX
        bounding_box1=bounding_box.cpu().numpy()
        labels1=labels.cpu().numpy()
        confidences=confidence.cpu().numpy()
        
        detections = [] 
        
        if(len(indices)!=0):
            for i in indices:
                if(labels1[i] == car_index):
                    x1,y1,x2,y2 = bounding_box1[i].astype('int')
                    x,y,w,h = [x1, y1,int(abs(x2-x1)),int(abs(y2-y1))]
                    detections.append(([x,y,w,h],confidences[i],'car'))
    
            tracks = object_tracker.update_tracks(detections, frame=frame)
            
            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                setIDs.add(track_id)
                ltrb = track.to_ltrb()
                bbox = ltrb
                cv2.rectangle(frame,(int(bbox[0]), int(bbox[1])),(int(bbox[2]), int(bbox[3])),(0,0,255),2)
                cv2.putText(frame, "ID: " + str(track_id), (int(bbox[0]), int(bbox[1] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            #Count 
#             count = object_tracker.tracker._next_id-1 
            count=len(setIDs)
            cv2.putText(frame, f'Count: {int(count)}', (20,100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)

            out.write(frame)
        
    out.release()


In [37]:
TrackCar(fasterrcnn,video,3,object_tracker,'/kaggle/working/VidhanaSoundha2.mp4')

# FasterRCNN + SORT

In [38]:
!pip install gitpython

In [39]:
import shutil
import git
import sys

#Uncomment like below if running it for 2nd time or more

# shutil.rmtree('/kaggle/working/sample1')

git.Repo.clone_from('https://github.com/KalyanRam1234/sort.git', '/kaggle/working/sample1')
sys.path.insert(0,'/kaggle/working/sample1/')

In [40]:
%pip install filterpy==1.4.5
# %pip install scikit-image==0.17.2
%pip install lap==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=b81046cd790c207cdb418d358674b4ddfd41cb52d7846143622a99e51f8ffe59
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.4 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for lap: filename=lap-0.4.0-cp310-cp310-linux_x86_64.whl size=1483137 sha256=3bb02d22c1a862e62f79b5461e4f18cc3dcd442c005249ec22707dde5fbd94db
  Stored in directory: /root/.cache/pip/wheels/00/42/2e/9dfe19270eea279d79e84767ff0d7b8082c3bf776cad00e83d
Successfully built lap
Note: you may need to restart the kernel to use updated packages.


In [67]:
from sort import *

In [80]:
video = cv2.VideoCapture('/kaggle/input/traffic-junction/Images/Images/Vidhana Soudha/video_20240211_142947.mp4')
object_tracker = Sort(max_age=600, min_hits=2)

In [81]:
def TrackCarSort(model,video, car_index,object_tracker,out_path):
    model.eval()
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), 15, (width, height))
    
    setIDs=set()
    while video.isOpened():
        # Read the next frame
        ret, frame = video.read()
        if not ret:
            break
            
        transform = transforms.ToTensor()
        img = transform(frame).to(device)
        with torch.no_grad():
          pred = model([img])
        

        #Creating Detection data for cars to pass into SORT tracks directly 
        
        bounding_box,labels,confidence = pred[0]["boxes"],pred[0]["labels"],pred[0]["scores"]
        indices = torch.argwhere(confidence>0.9)
        indices=indices.flatten()
        font = cv2.FONT_HERSHEY_COMPLEX
        bounding_box1=bounding_box.cpu().numpy()
        labels1=labels.cpu().numpy()
        confidences=confidence.cpu().numpy()
        
        detections = [] 
        
        if(len(indices)!=0):
            for i in indices:
                if(labels1[i] == car_index):
                    x1,y1,x2,y2 = bounding_box1[i].astype('int')
                    x,y,w,h = [int(x1), int(y1),int(abs(x2-x1)),int(abs(y2-y1))]
                    detections.append([int(x1),int(y1),int(x2),int(y2)])
            
            if(len(detections)==0): continue 

            detections=np.array(detections)
            tracks = object_tracker.update(detections)

            for id ,track in enumerate(tracks):
                bbox=track
                track_id=track[4]
                setIDs.add(track_id)
                cv2.rectangle(frame,(int(bbox[0]),int(bbox[1])),(int(bbox[2]),int(bbox[3])),(0,0,255),2)
                cv2.putText(frame, "ID: " + str(track_id), (int(bbox[0]), int(bbox[1] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            count=len(setIDs)
            cv2.putText(frame, f'Count: {int(count)}', (20,100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)

            out.write(frame)
#             break
        
    out.release()

In [82]:
TrackCarSort(fasterrcnn,video,3,object_tracker,'/kaggle/working/VidhanaSort2.mp4')